Discard: toy codes

In [1]:
import torch

In [ ]:
import swyft
import pylab as plt
import numpy as np

In [10]:
class NormalRatio(torch.nn.Module):
    def __init__(self, *shape, th = 5):
        super().__init__()
        self.log_factor = torch.nn.Parameter(torch.zeros(*shape))
        self.log_sigma = torch.nn.Parameter(torch.zeros(*shape))
        self.th = th
    
    def forward(self, z, m):
        if z.view(-1)[0] < 0:
            N = len(z)
            z[:] = self.sample(N, m)
        log_r = self.log_factor - 0.5*(z-m)**2/torch.exp(self.log_sigma)**2
        return log_r
    
    def sample(self, N, m):
        """Sample constrained priors"""
        sigma = torch.exp(self.log_sigma)
        width = torch.minimum(2*self.th*sigma, torch.ones_like(m))
        lo = torch.maximum(m-0.5*width, torch.zeros_like(m))
        lo = torch.minimum(lo, 1-width)
        return torch.rand(N, *m.shape, device=m.device)*width+lo

nr = NormalRatio(1)
m = torch.rand(1, 4)

In [11]:
class View:
    def __init__(self, **shapes):
        self._shapes = shapes
        self._i0 = {}
        self._i1 = {}
        
        N = 0
        for key, value in shapes.items():
            n = np.prod(value)
            self._i0[key] = N
            self._i1[key] = N + n
            N += n
        
        self._N = N
           
    def __getattr__(self, key):
        return self._u[..., self._i0[key]:self._i1[key]].view(*self._u.shape[:-1], *self._shapes[key])
    
    def __setattr__(self, key, value):
        if key[:1] == '_':
            object.__setattr__(self, key, value)
        else:
            self._u[self._i0[key]:self._i1[key]] = value.view(-1, np.prod(self._shapes[key]))
    
    def __call__(self, u):
        self._u = u
        return self
    
    def __len__(self):
        return self._N

In [8]:
class RatioSampler:
    def __init__(self):
        self.r1 = NormalRatio()
        self.x1_extractor = None
        self.features = lambda x: x
        self.get_m1 = nn.Linear(1, 10)
        self.r1 = NormalRatio(1)
        self.v = View(a = (100,), b = (1,))
    
    def embedding(self, x):
        return x
    
    def sample(self, x, N):
        z = x.new(N)*0-1
        self.forward(x, z)
        return z
    
    def forward(self, x, u):
        u = self.v(u)
        f = self.features(x)
        u = self.v(u)
        a = self.v(z).a
        b = self.v(z).b
        z1 = z[:, :N]
        z2 = z[:, N:]
        m1 = self.get_m1(f)
        r1 = self.r1(z['z1'], m1)
        m2 = self.get_m2(f)
        r2 = self.r2(z['z2'], m2)
        m3_bias = self.get_m3_bias(m2.detach())
        m3 = self.get_m3(f)
        self.r3(z.z3, m3 + z3_bias)
        
        r.freq = self.r1(z.freq, m3 + m3_bias)
        return r.freq

In [6]:
#u = torch.rand(100, 20)
v = View(a = (4, 5), b = (100,))
u = torch.rand(100, len(v))
v(u).a.shape

torch.Size([100, 4, 5])